# Assignment

here comes info about the assignment

## Import libraries

here come all the imported libraries

In [7]:
import pandas as pd
from pathlib import Path
import numpy as np
import math
import scipy
from scipy.signal import find_peaks
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd
#comma is added here
#comma added by ralf
#pleassseee
# rgjrgrtgjrtgjt5g

## Part I - Data import

Here comes the first part of the assignment. Importing the filtered data. Create a dataframe.
Show the data in a table.


In [8]:
file_path = 'who_ambient_air_quality_database_version_2023.csv'

df = pd.read_csv(file_path, sep = ';') 
# The file had an incorrect delimiter which caused a ParserError,
# this is solved by adding sep=';' when reading the file. 

C:\Users\HHapp\AppData\Local\Temp\ipykernel_17996\3021624361.py:3: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep = ';')


In [9]:
df.head(5)

,who_region,iso3,country_name,city,year,version,pm10_concentration,pm25_concentration,no2_concentration,pm10_tempcov,pm25_tempcov,no2_tempcov,type_of_stations,reference,web_link,population,population_source,latitude,longitude,who_ms
0,3_Sear,IND,India,Chennai,2018,version 2022,NaN,30.0,NaN,NaN,91.0,NaN,NaN,"U.S. Department of State, United States Enviro...",https://www.airnow.gov/index.cfm?action=airnow...,9890427,NaN,"13,08784","80,27847",1
1,3_Sear,IND,India,Solapur,2016,"version 2022, version 2018",NaN,39.0,NaN,NaN,99.0,NaN,NaN,"Central Pollution Control Board India, Environ...",NaN,985568,NaN,"17,6599188","75,9063906",1
2,3_Sear,IND,India,Chennai,2019,version 2022,NaN,39.0,NaN,NaN,85.0,NaN,NaN,"U.S. Department of State, United States Enviro...","[[[""EPA AirNow DOS"",""http://airnow.gov/index.c...",9890427,NaN,"13,08784","80,27847",1
3,3_Sear,IND,India,Hyderabad,2019,version 2022,NaN,42.0,NaN,NaN,87.0,NaN,NaN,"U.S. Department of State, United States Enviro...","[[[""EPA AirNow DOS"",""http://airnow.gov/index.c...",8943523,NaN,"17,38405","78,45636",1
4,3_Sear,IND,India,Pune,2017,version 2022,NaN,43.0,NaN,NaN,NaN,NaN,NaN,"Central Pollution Control Board India, Environ...",http://www.cpcb.gov.in/CAAQM/,5727530,NaN,"18,5053196","73,8238391",1


In [ ]:
df = df[df['reference'] == 'European Environmental Agency' ].drop_duplicates(subset=['country_name'])  #Filtered out all the non-european data
df

,who_region,iso3,country_name,city,year,version,pm10_concentration,pm25_concentration,no2_concentration,pm10_tempcov,pm25_tempcov,no2_tempcov,type_of_stations,reference,web_link,population,population_source,latitude,longitude,who_ms
36,4_Eur,ALB,Albania,Elbasan,2015,version 2023,NaN,NaN,24.0,NaN,NaN,98.0,1 Urban,European Environmental Agency,https://www.eea.europa.eu/themes/air/explore-a...,NaN,NaN,"41,111","20,0817",1
69,4_Eur,AUT,Austria,Am rechen,2013,version 2018,NaN,NaN,16.0,NaN,NaN,95.0,NaN,European Environmental Agency,https://www.eea.europa.eu/themes/air/explore-a...,3536,NaN,"48,21333333","15,19722222",1
332,4_Eur,BEL,Belgium,Anzegem,2010,version 2018,NaN,NaN,23.0,NaN,NaN,90.0,1 suburban,European Environmental Agency,https://www.eea.europa.eu/themes/air/explore-a...,14845,NaN,"50,833035","3,404508",1
501,4_Eur,BIH,Bosnia and Herzegovina,Goražde,2021,version 2023,NaN,26.0,NaN,NaN,87.0,NaN,1 Suburban,European Environmental Agency,https://www.eea.europa.eu/themes/air/explore-a...,NaN,NaN,"43,6616","18,9775",1
544,4_Eur,BGR,Bulgaria,Gulubovo,2016,version 2023,NaN,NaN,10.0,NaN,NaN,91.0,1 Urban,European Environmental Agency,https://www.eea.europa.eu/themes/air/explore-a...,NaN,NaN,"42,1465","25,8754",1
5558,4_Eur,HRV,Croatia,Dubrovnik,2014,version 2023,NaN,NaN,NaN,NaN,NaN,NaN,1 Suburban,European Environmental Agency,https://www.eea.europa.eu/themes/air/explore-a...,NaN,NaN,"42,6447","18,1226",1
5599,4_Eur,CYP,Cyprus,Larnaca,2013,version 2018,NaN,17.0,NaN,NaN,NaN,NaN,NaN,European Environmental Agency,https://www.eea.europa.eu/themes/air/explore-a...,51468,NaN,"34,91361237","33,61583328",1
5615,4_Eur,CZE,Czechia,Budisov nad budisovkou,2013,version 2018,NaN,NaN,8.0,NaN,NaN,96.0,1 Rural background,European Environmental Agency,https://www.eea.europa.eu/themes/air/explore-a...,3233,NaN,"49,77714129","17,54194671",1
5668,4_Eur,DNK,Denmark,Aalborg,2010,version 2018,NaN,17.0,27.0,NaN,90.0,90.0,4 urban,European Environmental Agency,https://www.eea.europa.eu/themes/air/explore-a...,192367,NaN,"57,049444","9,924178",1
5700,4_Eur,EST,Estonia,Kärla,2013,version 2018,NaN,5.0,2.0,NaN,93.0,99.0,1 Rural,European Environmental Agency,https://www.eea.europa.eu/themes/air/explore-a...,NaN,NaN,"58,3761","21,845",1


In [ ]:
# Maybe we should merge the data so that we have one row per country (instead of now per city within a country)
# We need to filter the data, get rid of columns that we will not look at.

## Part II - Data processing

Peak finding. Check of common peaks between data sets. 

## Part III - Data visualisation

Conclusion. Relate found info from Part II to our RQ. Perhaps with graphs, maps, etc. 